In [2]:
import keras
from keras.layers import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import multiprocessing as mp
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

df = pd.read_pickle('df_S300_featurized')
df = df.replace([np.inf, -np.inf, np.nan], 0)
df_X = df.drop('Input Data|S_300_atom',axis=1)
a = {}
for x in df_X.columns:
    if 'ChemEnvSiteFingerprint|GaussianSymmFunc' in x:
        a[x] = 'GaussianSymmFunc|'+x[39:]
df_X = df_X.rename(a,axis=1)

df_Y = df[['Input Data|S_300_atom']]

df_X=(df_X-df_X.min())/(df_X.max()-df_X.min())
#df_Y=(df_Y-df_Y.min())/(df_Y.max()-df_Y.min())
df_X = df_X.replace([np.inf, -np.inf, np.nan], 0)

f_test,_ = f_regression(df_X,df_Y)

def get_features(n):
    
    n_best = f_test.argsort()[-n:]
    
    return df_X.columns[n_best]

def CV_NN(model,df_X,df_Y,cv=5,epochs=200,batch_size=100,plot = False):
  df_X,df_Y = shuffle(df_X,df_Y)
  size = df_X.shape[0]
  mae_score = np.zeros(cv)
  map_score = np.zeros(cv)
  
  weights = model.get_weights()
  
  for i in range(cv):
    fit_params = {
            'x': df_X.values,
            'y': df_Y.values,
            'epochs': epochs+20,
            'batch_size': batch_size,
            'verbose': 0,
            'validation_split' : 1/cv
    }
    
    model.set_weights(weights)
    history = model.fit(**fit_params)
    mae =  history.history['mean_absolute_error']
    val_map = history.history['val_mean_absolute_percentage_error']
    val_mae = history.history['val_mean_absolute_error']   
    #score = np.sort(val_mae)[:40].mean()
    #score_p = np.sort(val_map)[:40].mean()
    score = np.array(val_mae[-40:]).mean()
    score_p = np.array(val_map[-40:]).mean()
    mae_score[i] = score
    map_score[i] = score_p
    temp = df_X.iloc[:size//cv,:]
    idx = temp.index
    df_X = df_X.drop(idx,axis=0)
    df_X = df_X.append(temp)
    temp = df_Y.loc[idx,:]
    df_Y = df_Y.drop(idx,axis=0)
    df_Y = df_Y.append(temp)
    if plot:
      fig, ax = plt.subplots(figsize=(6,6))
      ax.plot(mae,label='mae')
      ax.plot(val_mae,label='val_mae')
      # ax.set_ylim([0,6])
      ax.set_title('entropy @ 300K')
      ax.set_ylabel('MAE [J/Kmol]')
      ax.set_ylabel('epochs')
      ax.legend()
      #fig.savefig("e{}.jpg".format(str(T)))
      ax.grid()
      
  return (mae_score.mean(), map_score.mean())

def gen_model(n_features,bias=2):
    f_input = Input(shape=(n_features,))
    hidden1 = Dense(int(n_features/2)+bias,activation='relu')(f_input)
    #hidden2 = Dense(int(n_features/2)+2,activation='relu')(hidden1)
    out = Dense(1,activation='linear')(hidden1) #output is bounded below
    model = keras.models.Model(f_input,out)
    model.compile(loss = 'mae', metrics=['mae','mean_absolute_percentage_error'],
                  optimizer=keras.optimizers.Adam(0.01))
    return model
  
    
def grid_search(df_X,df_Y,n_features):
    iterations = len(n_features)
    i=1
    mae_score = []
    map_score = []
    for n in n_features:
        print('{}/{}'.format(i,iterations))
        features = get_features(n)
        X = df_X[features]
        Y = df_Y
        model = gen_model(n,bias=16)
        best_score, best_scorep = CV_NN(model,X,Y)
        print("{} features, {:.2f}% map/ {:.2f} mae".format(n,best_scorep,best_score))
        mae_score += [best_score]
        map_score += [best_scorep]
        i+=1
    return (mae_score,map_score)

Using TensorFlow backend.
C:\Users\Pierre-Paul\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pierre-Paul\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\Pierre-Paul\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Pierre-Paul\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Pierre-Paul\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 &

In [3]:
n_features = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 24, 28, 32, 35, 40, 45, 50, 55, 60,
      70, 80, 90, 100, 150, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1100]
mae_err, map_err = grid_search(df_X,df_Y,n_features=n_features)

1/38
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
2 features, 60.16% map/ 11.07 mae
2/38
3 features, 60.10% map/ 11.09 mae
3/38
4 features, 60.20% map/ 11.10 mae
4/38
5 features, 59.73% map/ 11.06 mae
5/38
6 features, 59.76% map/ 11.06 mae
6/38
7 features, 59.89% map/ 11.04 mae
7/38
8 features, 59.91% map/ 11.06 mae
8/38
9 features, 59.71% map/ 11.09 mae
9/38
10 features, 59.90% map/ 11.04 mae
10/38
12 features, 59.83% map/ 11.06 mae
11/38
14 features, 60.13% map/ 11.09 mae
12/38
16 features, 59.30% map/ 11.06 mae
13/38
18 features, 59.91% map/ 11.09 mae
14/38
20 features, 60.12% map/ 11.05 mae
15/38
24 features, 60.09% map/ 11.06 mae
16/38
28 features, 59.91% map/ 11.10 mae
17/38
32 features, 59.72% map/ 11.06 mae
18/38
35 features, 60.21% map/ 11.08 mae
19/38
40 features, 59.52% map/ 11.06 mae
20/38
45 features, 60.04% map/ 11.08 mae
21/38
50 features, 59.83% map/ 11.08 mae
22/38
55 features, 60.33% map/ 11.09

In [5]:
err = np.array(mae_err)
np.save("corr_fit_results",err)